# A polynomial invariant of oriented knotted 4-valent graphs

Knotted 4-valent graphs arise as components of the quantum states in loop quantum gravity (LQG). In particular, the states are diffeomorphism classes of graphs embedded in a three-dimensional manifold; the edges of the graphs are colored with $SU(2)$ representations. Graphs with valence four are natural candidates for physical states, since they are dual in some sense to three-dimensional manifold triangulations. In addition, there are reasons within the theory for choosing 4-valent graphs. The LQG volume operator only gives non-trivial eigenvalues when the valence is four or greater. On the flip slide, vertices of valence five or higher have additional degrees of freedom not taken into account by the intertwiner between the $SU(2)$ representations when the diffeomorphism group acts on all points in the embedding manifold equally (Grot and Rovelli, [arXiv:gr-qc/9604010](https://arxiv.org/abs/gr-qc/9604010)).

This SageMath notebook is intended to take a DT sequence for an oriented knotted 4-valent graph with rigid vertices, and produce the associated graph polynomial. This is done in a few steps:

1. Find the orientation of the graph nodes by running the DT algorithm on the sequence, giving the $f(i)$ function on node labels $i$. The convention is that $f(i) = +1$ if the other arc crosses from right to left, and $f(i) = -1$ if the other arc crosses from left to right.
1. Once the orientation is known, it is possible to use the graph polynomial skein relation to find the polynomial in terms of sums of polynomials in the variables $c, r, s$, each multiplied by a link. The skein relation for the graph polynomial shown below acts on a vertex.
![](./img/Graph-poly-01.png)
Here, the 4-valent vertex on the left in the projection plane is shown using the notation of Wan ([arXiv:0710.1312](https://arxiv.org/abs/0710.1312)). The variable coefficients can be remembered by ($c$)rossing, ($r$)everse crossing, and ($s$)plit. Taking the mirror image of this relation -- i.e. switching the crossing and vertex type as appropriate -- one obtains the skein relation when the "upper" edge of the vertex passes from left to right.
1. These links can then be evaluated to find their link polynomials; here, the Jones polynomial $X(A)$ is used.

Define the DT sequence for the graph. Remember that this should be a Python list of three element lists. The first element is an even number, the second an odd; these are the two labels for that particular node. The third element gives the crossing or vertex type: -2 for a vertex with the upper edge on the odd label (label superscript $\ell$), -1 for a crossing with the upper edge on the odd label (label superscript $-$), and the positive numbers are the flipped cases (superscripts $u$ and $+$).

For example, the graph sequence $3^\ell 5^u 7^+ 1^-$ corresponds to the Python list

```python
    seq = [[0, 3, -2], [2, 5, 2], [4, 7, 1], [6, 1, -1]]
```

The next cell defines the graph polynomial variables used in Sage.

In [ ]:
var('c, r, s')

The following imports the function `isRealizable()` for the DT sequence `seq`, which gives the sequence orientation information $f(i)$. For a list `seq` corresponding to a realizable DT sequence, calling `isRealizable(seq, f_list = True)` will give a list of $f(i)$ values for each node label $i$. **Note:** Evidently, Jupyter notebooks require a beginning capital letter for the name of the folder from which the module is imported.

In [ ]:
from Graphmaker.isRealizable import isRealizable

The next command creates the orientation list `orientList` for the DT sequence `seq`.

In [ ]:
# Example sequence 3^u 5^u 7^u 1^u

seq = [[0, 3, 2], [2, 5, 2], [4, 7, 2], [6, 1, 2]]

orientList = isRealizable(seq, f_list = True)
orientList

The function `planarDiagram` takes the DT sequence and orientation lists, and produces a planar diagram list from them. This will be used in the actual algorithm to find the graph polynomial.

In [ ]:
from Graphmaker.planarDiagram import planarDiagram

Here, we show the planar diagram notation for $3^u 5^u 7^u 1^u$, where the orientation is that found above in a previous cell.

In [ ]:
planarDiagram(seq, orientList)

Note that $f(i)$ has now been encoded in the second list returned by `planarDiagram()`. In particular, for every node in the graph, there is an element in the second list, which is positive if the incoming lower edge sees the other edge passing right-to-left, and negative otherwise. This will not be important for the crossings, but will be used in the function `graphPoly()` below which uses the polynomial skein relation.

Before we get into solving for the graph polynomials, we need an additional function, that relabels the edge labels to replace any missing numbers that have been lost in the simplification process. Note: do **not** include a sorting part to the list that is returned, since the nodes are treated in a certain order by the `graphPoly()` algorithm.

In [ ]:
from Graphmaker.reorder import reorder

Here is the main code -- the function `graphPoly` takes a graph in PD notation, along with its list of $f(i)$ values, and returns the associated graph polynomial.

In [ ]:
def graphPoly(pd_list, type_list):
    """
    This is a recursive function to find the graph
    polynomial associated with the graph described
    by the PD list pd_list and the orientation list
    type_list. When the algorithm is complete, it
    will return the graph polynomial.
    """
    
    # If the graph has been completely reduced down
    # to a link, return the link invariant polynomial,
    # using any choice of such invariant
    
    if type_list == []:
        L = Link(pd_list)
        return L.jones_polynomial(skein_normalization = True, algorithm = 'statesum')
    
    # If the graph has not been completely reduced,
    # work on last node; this will be deleted from the
    # end of type_list, but kept in pd_list, so the
    # link invariant can be calculated.
    
    # If the node type is a crossing, return the function
    # for the remainder of the graph, without the given
    # crossing
    
    if abs(type_list[-1]) == 1:
        return graphPoly([node for node in pd_list], type_list[:-1])
    
    # If the node type is a vertex, use the graph
    # polynomial skein relation.
    
    elif abs(type_list[-1]) == 2:
    
        # Create new function to return

        new_func = 0
        
        # Which node are we working with? Given by length of
        # current type_list
        
        node_index = len(type_list) - 1
        
        #--------------------------------------------------------------------#
        
        # Case (c)rossing: Convert the vertex into an equivalent crossing, i.e.
        # let the upper edge of the vertex now be the upper edge of the crossing.
        # This does not affect the PD notation at all, so just use copy of the
        # current pd_list. No need to reorder the list, since the assumption is
        # that it is already well-ordered, and no nodes have been removed.
        
        new_func += c * graphPoly([node for node in pd_list], type_list[:-1])
        
        #--------------------------------------------------------------------#
        
        # Case (r)eversed crossing: Create a crossing, but with the lower edge
        # of the vertex becoming the upper edge of the crossing. This moves the
        # incoming lower edge of the crossing to what was the incoming edge of
        # the upper vertex edge. If the former upper edge was right-to-left,
        # then the PD notation for the crossing (a0, a1, a2, a3) -> (a1, a2, a3, a0);
        # if the former upper edge was left-to-right, then (a0, a1, a2, a3) ->
        # (a3, a0, a1, a2). Hopefully this information is correctly stored in
        # type_list[node_index], with +2 if the upper edge is right-to-left, and
        # -2 if left-to-right. No need to reorder the list, since the assumption is
        # that it is already well-ordered, and no nodes have been removed.
        
        if type_list[node_index] == 2:
            change = 1
        elif type_list[node_index] == -2:
            change = -1
            
        new_pd_list = [pd_list[iii] if iii != node_index
                       else [pd_list[node_index][(iii + change) % 4] for iii in range(4)]
                       for iii in range(len(pd_list))]
        
        new_func += r * graphPoly(new_pd_list, type_list[:-1])
        
        #--------------------------------------------------------------------#
        
        # Case (s)plit: Connect the two incoming edges to their neighboring
        # outgoing edges, as appropriate for the vertex type. This is the only
        # case where having two edges of the vertex be the same is an issue.
        # However, all that it means is that an unlink must be added to the
        # graph; the vertex PD notation can be directly converted to a crossing,
        # which is equivalent to the actual situation by a RI move. The special
        # cases all boil down to whether adjacent edges are the same, so that
        # is the only check required. For the generic case, the PD notation is
        # reordered, since a node is removed.
        
        [a0, a1, a2, a3] = pd_list[node_index]
        
        if a0 == a1 or a1 == a2 or a2 == a3 or a3 == a0:
                
            # Find largest current edge label in new graph list
                
            high_label = max([max(node) for node in new_pd_list])
            
            # Find new split term in graph polynomial, for current PD list
            # plus a new unlink, with edge labels greater than all previous
            # edge labels
            
            new_func += s * graphPoly([node for node in pd_list] + [[high_label + 1, high_label + 1, high_label + 2, high_label + 2]], type_list[:-1])
            
        else:
            
            # Generic case
            
            if type_list[node_index] == 2:         # Right-to-left crossing
            
                # Find max and min labels for working vertex, for changing
                # max(a0, a3) -> min(a0, a3), max(a1, a2) -> min(a1, a2)

                first_max, first_min = max(a0, a3), min(a0, a3)
                second_max, second_min = max(a1, a2), min(a1, a2)
            
            elif type_list[node_index] == -2:      # Left-to-right crossing
            
                # Find max and min labels for working vertex, for changing
                # max(a0, a1) -> min(a0, a1), max(a2, a3) -> min(a2, a3)

                first_max, first_min = max(a0, a1), min(a0, a1)
                second_max, second_min = max(a2, a3), min(a2, a3)
            
            # Create new PD list, removing current vertex, and changing all
            # max edge labels to min edge labels, as appropriate for split
            
            new_pd_list = [[node[iii] if node[iii] != first_max else first_min for iii in range(4)]
                           for node in pd_list if node != pd_list[node_index]]
            
            new_pd_list = [[node[iii] if node[iii] != second_max else second_min for iii in range(4)]
                           for node in new_pd_list]
            
            # Find next term in graph polynomial
            
            new_func += s * graphPoly(reorder([node for node in new_pd_list]), type_list[:-1])
        
        #--------------------------------------------------------------------#
        
        # Return resulting polynomial
        
        return new_func

Here are some test cases; in the list below, it is assumed that $f(0) = +1$, i.e. at the node labeled with 0, the cross-edge is right-to-left.

1. $1^\ell$: `-s*A^-2 + (c + r) - s*A^2`
1. $3^\ell 1^\ell, 3^\ell 1^u$: `s^2*A^-4 - (2*c*s + 2*r*s)*A^-2 + (c^2 + 2*c*r + r^2 + 2*s^2) - (2*c*s + 2*r*s)*A^2 + s^2*A^4`
1. $3^\ell 5^+ 1^+$: `-r*A^-16 + r*A^-12 - s*A^-10 + r*A^-4 - s*A^-2 + c`
1. $3^\ell 5^- 1^-$: `r - s*A^2 + c*A^4 - s*A^10 + c*A^12 - c*A^16`
1. $3^\ell 5^- 1^u$: `-(c*s + r*s)*A^-2 + (c^2 + c*r + r^2 + s^2) - (2*c*s + 2*r*s)*A^2 + c*r*A^4 - (c*s + r*s)*A^10 + c*r*A^12 - c*r*A^16`
1. $3^\ell 5^\ell 1^+$: `-r^2*A^-16 + r^2*A^-12 - 2*r*s*A^-10 + r^2*A^-4 - (2*c*s + 2*r*s)*A^-2 + (c^2 + 2*c*r + s^2) - 2*c*s*A^2`
1. $3^\ell 5^\ell 1^-$: `-2*r*s*A^-2 + (2*c*r + r^2 + s^2) - 2(*c*s + r*s)*A^2 + c^2*A^4 - 2*c*s*A^10 + c^2*A^12 - c^2*A^16`
1. $3^\ell 5^u 1^u$: `-c^2*r*A^-16 + c^2*r*A^-12 + (-c^2*s - 2*c*r*s*A^-10 + c^2*r*A^-4 + (-3*c^2*s - 4*c*r*s - 2*r^2*s - s^3)*A^-2 + (c^3 + 2*c^2*r + 2*c*r^2 + r^3 + 3*c*s^2 + 3*r*s^2) + (-2*c^2*s - 4*c*r*s - 3*r^2*s - s^3*A^2 + c*r^2*A^4 + (-2*c*r*s - r^2*s)*A^10 + c*r^2*A^12 - c*r^2*A^16`
1. $3^\ell 5^\ell 1^u$: `-c*r^2*A^-16 + c*r^2*A^-12 + (-2*c*r*s - r^2*s)*A^-10 + c*r^2*A^-4 + (-2*c^2*s - 4*c*r*s - 3*r^2*s - s^3)*A^-2 + (c^3 + 2*c^2*r + 2*c*r^2 + r^3 + 3*c*s^2 + 3*r*s^2) + (-3*c^2*s - 4*c*r*s - 2*r^2*s - s^3)*A^2 + c^2*r*A^4 +(-c^2*s - 2*c*r*s)*A^10 + c^2*r*A^12 - c^2*r*A^16` (this is the same as applying $A \to A^{-1}$ to the polynomial of the previous graph, but they are *not* mirror images?)
1. $3^\ell 5^\ell 1^\ell$: `-r^3*A^-16 + r^3*A^-12 - 3*r^2*s*A^-10 + r^3*A^-4 + (-6*c*r*s - 3*r^2*s - s^3)*A^-2 + (3*c^2*r + 3*c*r^2 + 3*c*s^2 + 3*r*s^2) + (-3*c^2*s - 6*c*r*s - s^3)*A^2 + c^3*A^4 -3*c^2*s*A^10 + c^3*A^12 - c^3*A^16`

Here is a snippet of code that allows one to start from the DT notation for a knotted 4-valent graph, and find the corresponding graph polynomial, using the Jones polynomial for the knot invariant. Unless otherwise changed by the user, the assumption is that $f(0) = +1$.

In [ ]:
# DT notation for graph

seq = [[0, 3, -2], [2, 5, -2], [4, 7, -1], [6, 1, 1]]

# Use isRealizable() to solve for values
# of f(i) for each node label

orientList = isRealizable(seq, f_list = True)
print(orientList)

# Show PD notation for given graph

print(planarDiagram(seq, orientList))

# Find graph polynomial, and show list
# of coefficients for variable A coming
# from Jones polynomial

function = graphPoly(*planarDiagram(seq, orientList))
function.coefficients()

The code below finds all sequences from `Nmin` to `Nmax` number of nodes, and reports which are new knotted graphs. First, the following modules must be imported.

In [ ]:
from functools import reduce
from Graphmaker.createSequences import createSequences
from Graphmaker.modOrbit import modOrbit
from time import time

Then the cell below contains the actual code, which finds all possible ways to put crossings in the diagram without having a situation which can be easily removed by a RII graph move; there is no need for checking RI moves, since they would not appear with prime knotted graph. This version of the code is rather rough, and can be improved by, e.g. creating a priority queue, where graphs with one additional crossing are added only if the original graph has not already been found. This would eliminate repeated graphs, since none of the graphs found by adding crossings would not be original if the first one was not.

In [ ]:
Nmin = 3                            # Minimum, maximum number of nodes for graphs
Nmax = 5

# Store results in two dictionaries, where
# keys are (numNodes, numVert), and value is
# either the graph DT notation, or else the
# graph polynomial for that graph

graphSeqDict = {iii : [] for iii in range(1, Nmax + 1)}
graphPolyDict = {iii : [] for iii in range(1, Nmax + 1)}

# Keep count of number of graphs with
# given number of nodes, vertices

numGraphs = [[0 for iii in range(Nmax + 1)] for jjj in range(Nmax + 2)]

# Run through range of nodes

for N in range(Nmin, Nmax + 1):     # Number of nodes (vertices + crossings) in graph
    
    startTime = time()
    
    # Define final list of graphs for each number of nodes
    
    finalGraphList = []
    
    # Create all possible lists of length N lists with entries of +/- 1 (for
    # a crossing) or +/- 2 (for a vertex)
    
    signList = []
    
    for iii in range(4 ** N):
        tempList = [0 for jjj in range(N)]
        kkk = 0
        
        while kkk < N:
            if iii % 4 == 0:
                tempList[kkk] = -2      # Vertex, upper edge on odd label
            elif iii % 4 == 1:
                tempList[kkk] = -1      # Crossing, upper edge on odd label
            elif iii % 4 == 2:
                tempList[kkk] = 1       # Crossing, upper edge on even label
            else:
                tempList[kkk] = 2       # Vertex, upper edge on even label
                
            iii = iii // 4
            kkk += 1
            
        # If tempList is all crossings, do not include in signList
            
        if abs(reduce((lambda x, y: x * y), tempList)) != 1:
            signList += [tempList]
            
    # Create sequences of length N, and then for each such sequence, add
    # crossing/vertex information to each node, and see if each of these new
    # sequences are isomorphic to previous results; if not, include in
    # finalGraphList.

    processList = createSequences(N)
    
    for nextGraph in processList:
        
        # Go through all possible node types from signList, add to nextGraph,
        # and see if this graph is isomorphic to a previous graph
        
        for signChoice in signList:
            
            # Check to see if RII move can be used to remove this choice from
            # the queue; unnecessary to check RI moves, since these would not
            # appear in a prime graph sequence. Here are the sub-sequences to
            # check for, with c = +/- 1:
            #
            #   (i, j, c)(j + 1, i + 1, -c)     case (1)
            #   (i, j, c)(j + 1, i - 1, -c)     case (2)
            #   (i, j, c)(j - 1, i - 1, -c)     case (3)
            #   (i, j, c)(j - 1, i + 1, -c)     case (4)
            
            tempGraph = [nextGraph[iii] + [signChoice[iii]] for iii in range(N)]
            
            flag = False
            
            for nodeIndex in range(N):
                iii, jjj = tempGraph[nodeIndex][0], tempGraph[nodeIndex][1]
                
                # Cases (1) and (2)
                
                jjjIndex = ((jjj + 1) % (2 * N)) // 2
                
                if abs(tempGraph[jjjIndex][1] - iii) == 1 and tempGraph[nodeIndex][2] * tempGraph[jjjIndex][2] == -1:
                    flag = True
                
                # Cases (3) and (4)
                
                jjjIndex = ((jjj - 1) % (2 * N)) // 2
                
                if abs(tempGraph[jjjIndex][1] - iii) == 1 and tempGraph[nodeIndex][2] * tempGraph[jjjIndex][2] == -1:
                    flag = True
            
            if flag:
                continue
            
            # If RII test passed, go through all isomorphic sequences, find
            # lowest order sequence, and see if it has already been obtained
            
            tempGraph = modOrbit(tempGraph, 2 * N, crossing = True)
            
            if tempGraph not in finalGraphList:
                finalGraphList += [tempGraph]
    
    # Find out whether each graph obtained is equivalent to any previous
    # graphs using the graph polynomial; if not, store graph and its
    # polynomial in graphSeqDict and graphPolyDict
    
    rejected = 0
    
    for graph in finalGraphList:
        
        # Find graph polynomial
        
        function = graphPoly(*planarDiagram(graph, isRealizable(graph, f_list = True)))
        
        # Find number of vertices in graph
            
        numVert = 0

        for vertex in graph:
            if abs(vertex[2]) == 2:
                numVert += 1
                
        # Go to key numVert in graphPolyDict
        # and see if current graph polynomial is
        # there; if not, add to both dicts
        
        if function in graphPolyDict[numVert]:
            #print 'Rejected:', graph, '->', graphSeqDict[numVert][graphPolyDict[numVert].index(function)]
            rejected += 1
        else:
            
        #if function not in graphPolyDict[numVert]:
            graphSeqDict[numVert] += [graph]
            graphPolyDict[numVert] += [function]
                    
            # Record graph in numGraphs
            
            numGraphs[N][numVert] += 1
            
    
    print 'Rejected # =', rejected
    print(time() - startTime)
    print('===')
    
# Print number of graphs by number of
# nodes and vertices
    
for numNodes in range(Nmax + 1):
    print(numGraphs[numNodes])